In [1]:
import pandas as pd
import numpy as np
import os

NOTEBOOK_DIR = os.getcwd()
project_dir = os.path.dirname(NOTEBOOK_DIR)
apps_dir = os.path.join(project_dir, 'apps')
home_dir = os.path.join(apps_dir, 'home')
static_dir = os.path.join(apps_dir, 'static')
DATA_DIR = os.path.join(static_dir, 'assets', 'data')

### Load Site design Excel file and export as pickel file for faster operation

In [3]:
SITE_DESIGN_FN = os.path.join(NOTEBOOK_DIR, 'field_design.xlsm')
SITE_DESIGN = [
    ['sidebar'],
    ['page','page_element'],
    ['meta_form'],
    ['form','form_field'],
    ['model','model_field'],
    ['table','table_field']
]
SITE_DESIGN_SHEETS = ['sidebar','page','page_element','meta_form','form','form_field','model','model_field','table','table_field']
int_cols = [[],[],[],[],[],['Size'],[],[],[],[]]

def convert_cols(df, _int_cols):
    df = df.sort_values(by='#').drop('#', axis=1)
    cols = df.columns.tolist()
    str_cols = [c for c in cols if c not in _int_cols]
    dt_dct = {c:np.dtype('i1') for c in _int_cols}
    dt_dct.update({c:np.dtype('str') for c in str_cols})
    df[_int_cols] = df[_int_cols].fillna(-1)
    df[str_cols] = df[str_cols].fillna('')
    df = df.astype(dt_dct)
    return df

def export_pickles():
    for _sheet, _int_cols in zip(SITE_DESIGN_SHEETS, int_cols):
        convert_cols(
            pd.read_excel(
                SITE_DESIGN_FN,
                sheet_name=_sheet
            ),
            _int_cols
        ).to_pickle(
            os.path.join(DATA_DIR, f'{_sheet}.pkl')
        )
        print('Exported PKL: ', f'{_sheet}.pkl')
    return

export_pickles()

Exported PKL:  sidebar.pkl
Exported PKL:  page.pkl
Exported PKL:  page_element.pkl
Exported PKL:  meta_form.pkl
Exported PKL:  form.pkl
Exported PKL:  form_field.pkl
Exported PKL:  model.pkl
Exported PKL:  model_field.pkl
Exported PKL:  table.pkl
Exported PKL:  table_field.pkl


### Print models.py code generated from site design

In [14]:
def model_str():    
    export_pickles()
    code_list = ['']
    models = pd.read_pickle(os.path.join(DATA_DIR, f'model.pkl'))
    model_fields = pd.read_pickle(os.path.join(DATA_DIR, f'model_field.pkl'))
    for _, m in models.iterrows():
        fields = "\n".join([f'    {f.Code}'for _, f in model_fields[model_fields.Model == m.Model].iterrows()])
        if fields == '':
            fields = "    # No Fields Defined"
        code = "\n".join([
            f"\nclass {m.Model}({m.Extends}):",
            f"{fields}",
            f'    def __str__(self):',
            f"        return f'{m.String}'"
        ])
        code_list += [code]
    s = "\n".join(code_list)
    pd.DataFrame([s]).to_clipboard(index=False, header=False, quotechar='\t')
    return
    
model_str()

Exported PKL:  sidebar.pkl
Exported PKL:  page.pkl
Exported PKL:  page_element.pkl
Exported PKL:  meta_form.pkl
Exported PKL:  form.pkl
Exported PKL:  form_field.pkl
Exported PKL:  model.pkl
Exported PKL:  model_field.pkl
Exported PKL:  table.pkl
Exported PKL:  table_field.pkl


In [15]:
def form_str():    
    export_pickles()
    code_list = ['']
    forms = pd.read_pickle(os.path.join(DATA_DIR, f'form.pkl'))
    form_fields = pd.read_pickle(os.path.join(DATA_DIR, f'form_field.pkl'))
    for _, f in forms.iterrows():
        fields = "', '".join(form_fields[form_fields.Form == f.Form].Field.tolist())
        code = "\n".join([
            f"\nclass {f.Form}({f.Extends}):",
            f"    class Meta:",
            f"      model = {f.Model}",
            f"      fields = ['{fields}']"
        ])
        code_list += [code]
    s = "\n".join(code_list)    
    pd.DataFrame([s]).to_clipboard(index=False, header=False, quotechar='\t')
    return

form_str()

Exported PKL:  sidebar.pkl
Exported PKL:  page.pkl
Exported PKL:  page_element.pkl
Exported PKL:  meta_form.pkl
Exported PKL:  form.pkl
Exported PKL:  form_field.pkl
Exported PKL:  model.pkl
Exported PKL:  model_field.pkl
Exported PKL:  table.pkl
Exported PKL:  table_field.pkl


In [38]:
class Test(object):
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

thisObj = Test('var1','var2','var3')
thisdict = {
    'object': thisObj
}

thisdict['object'].a

'var1'

In [53]:
id = 'bill'

id or 'name'

'bill'